In [5]:
import numpy as np
from scipy.io import loadmat
import shutil
from imutils import paths
import random
import os

#### Đối với chuẩn bị dữ liệu cho Transfer learning : Feature Extractor thì gom vào folder nhãn trong các folder train , test , valid vào 1 folder là được 

#### Chuẩn bị dữ liệu cho Code Model Implement và Transfer learning : Fine tuning 

##### Rename toàn bộ ảnh trong folder 102flowers

Label

In [ ]:
# file_path = 'Dataset\imagelabels.mat'
# mat_data = loadmat(file_path)
# # print(mat_data)
# labels = mat_data['labels']
# print(len(labels[0]))
# print(labels[0])
# print(np.unique(labels))

Thêm r nếu không nó sẽ hiểu \102 là một kí tự đặt biệt 

In [41]:
import os

# Đường dẫn đến thư mục chứa các ảnh
folder_path = r"Dataset\102flowers\jpg" # thêm r nếu không nó sẽ hiểu 

# Đọc nhãn từ file imagelabels.mat
file_path = 'Dataset\imagelabels.mat'
mat_data = loadmat(file_path)
labels = mat_data['labels'][0]  # Lấy danh sách nhãn, chuyển về mảng 1D

# Lặp qua từng tệp trong thư mục
index = 0 
for filename in os.listdir(folder_path):
    # Tạo đường dẫn đầy đủ đến tệp
    filepath = os.path.join(folder_path, filename)
    print(filepath)
    # Đổi tên tệp bằng cách thêm nhãn phía trước
    new_filename = f"{labels[index]}label_{filename}"
    # Đổi tên tệp
    os.rename(filepath, os.path.join(folder_path, new_filename))
    index = index + 1
print("Đã đổi tên tất cả các tệp trong thư mục!")

Dataset\102flowers\jpg\image_00001.jpg
Dataset\102flowers\jpg\image_00002.jpg
Dataset\102flowers\jpg\image_00003.jpg
Dataset\102flowers\jpg\image_00004.jpg
Dataset\102flowers\jpg\image_00005.jpg
Dataset\102flowers\jpg\image_00006.jpg
Dataset\102flowers\jpg\image_00007.jpg
Dataset\102flowers\jpg\image_00008.jpg
Dataset\102flowers\jpg\image_00009.jpg
Dataset\102flowers\jpg\image_00010.jpg
Dataset\102flowers\jpg\image_00011.jpg
Dataset\102flowers\jpg\image_00012.jpg
Dataset\102flowers\jpg\image_00013.jpg
Dataset\102flowers\jpg\image_00014.jpg
Dataset\102flowers\jpg\image_00015.jpg
Dataset\102flowers\jpg\image_00016.jpg
Dataset\102flowers\jpg\image_00017.jpg
Dataset\102flowers\jpg\image_00018.jpg
Dataset\102flowers\jpg\image_00019.jpg
Dataset\102flowers\jpg\image_00020.jpg
Dataset\102flowers\jpg\image_00021.jpg
Dataset\102flowers\jpg\image_00022.jpg
Dataset\102flowers\jpg\image_00023.jpg
Dataset\102flowers\jpg\image_00024.jpg
Dataset\102flowers\jpg\image_00025.jpg
Dataset\102flowers\jpg\im

##### Lấy ra top 15 hoa có số ảnh nhiều nhất , chỉ train 15 loại hoa này 

In [59]:
import numpy as np

file_path = 'Dataset\imagelabels.mat'
mat_data = loadmat(file_path)
# print(mat_data)
labels = mat_data['labels']
print(len(labels[0]))
print(labels[0])
print(np.unique(labels))

unique_labels, counts = np.unique(labels[0], return_counts=True)
sorted_indices = np.argsort(counts)[::-1]
top_labels = unique_labels[sorted_indices][:15]
top_counts = counts[sorted_indices][:15]
i = 1
total = 0
for label, count in zip(top_labels, top_counts):
    print(f"{i}. {label}: {count}")
    total += count
    i += 1

print('Have',total,'img of 15 label')
# LƯU Ý : NẾU LẤY RA TOP 15 THÌ CHỈ CẦN THÊM LỆNH 
labels = top_labels

8189
[77 77 77 ... 62 62 62]
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102]
1. 51: 258
2. 77: 251
3. 46: 196
4. 73: 194
5. 89: 184
6. 74: 171
7. 81: 166
8. 94: 162
9. 88: 154
10. 78: 137
11. 83: 131
12. 43: 130
13. 95: 128
14. 41: 127
15. 75: 120
Have 2509 img of 15 label


##### Chia dữ liệu train , valid , test 

In [58]:
unique_labels = np.unique(labels).astype(str)
new_unique_labels = []
for label in unique_labels:
    new_label = label + 'label'
    new_unique_labels.append(new_label)
new_unique_labels = np.array(new_unique_labels)
print(new_unique_labels)

['41label' '43label' '46label' '51label' '73label' '74label' '75label'
 '77label' '78label' '81label' '83label' '88label' '89label' '94label'
 '95label']


TOP 15 flower sẽ là 2509 ảnh 
Toàn bộ sẽ là 8189 ảnh 

In [60]:
import os
import shutil
import random
import zipfile

data_dir = r'Dataset/102flowers/jpg'
# root_dir = 'Dataset/Dataset_102flower_8189'
root_dir = 'Dataset/Dataset_102flower_TOP15'

# Đường dẫn tới thư mục con 'train', 'val' và 'test'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')

# Tạo các thư mục con 'train', 'val' và 'test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Danh sách các nhãn
labels = new_unique_labels

train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Lặp qua các nhãn
for label in labels:
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)

    # Đường dẫn tới các tệp thuộc nhãn hiện tại
    label_files = [file for file in os.listdir(data_dir) if file.startswith(label)]
    print(data_dir)
    print(label_files)
    
    # Sắp xếp ngẫu nhiên các tệp
    random.shuffle(label_files)

    # Số lượng tệp trong từng thư mục con
    num_files = len(label_files)
    num_train = int(num_files * train_ratio)
    num_val = int(num_files * val_ratio)
    num_test = num_files - num_train - num_val

    # Chia các tệp vào các thư mục con tương ứng
    train_files = label_files[:num_train]
    val_files = label_files[num_train:num_train + num_val]
    test_files = label_files[num_train + num_val:]

    for file in train_files:
        src = os.path.join(data_dir, file)
        dst = os.path.join(train_dir, label, file)
        shutil.copy2(src, dst)

    for file in val_files:
        src = os.path.join(data_dir, file)
        dst = os.path.join(val_dir, label, file)
        shutil.copy2(src, dst)

    for file in test_files:
        src = os.path.join(data_dir, file)
        dst = os.path.join(test_dir, label, file)
        shutil.copy2(src, dst)


Dataset/102flowers/jpg
['41label_image_02189.jpg', '41label_image_02190.jpg', '41label_image_02191.jpg', '41label_image_02192.jpg', '41label_image_02193.jpg', '41label_image_02194.jpg', '41label_image_02195.jpg', '41label_image_02196.jpg', '41label_image_02197.jpg', '41label_image_02198.jpg', '41label_image_02199.jpg', '41label_image_02200.jpg', '41label_image_02201.jpg', '41label_image_02202.jpg', '41label_image_02203.jpg', '41label_image_02204.jpg', '41label_image_02205.jpg', '41label_image_02206.jpg', '41label_image_02207.jpg', '41label_image_02208.jpg', '41label_image_02209.jpg', '41label_image_02210.jpg', '41label_image_02211.jpg', '41label_image_02212.jpg', '41label_image_02213.jpg', '41label_image_02214.jpg', '41label_image_02215.jpg', '41label_image_02216.jpg', '41label_image_02217.jpg', '41label_image_02218.jpg', '41label_image_02219.jpg', '41label_image_02220.jpg', '41label_image_02221.jpg', '41label_image_02222.jpg', '41label_image_02223.jpg', '41label_image_02224.jpg', '41l